In [1]:
import pandas as pd
import numpy as np
import datetime
import re
import random
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from tqdm import tqdm
from sklearn.metrics import f1_score

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

/Users/bondarchukgleb/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
np.__version__

'1.26.2'

In [3]:
df = pd.read_csv("train_data_sila/train_data.csv")
df_problem_type = pd.read_csv("synthetic_dataset_for_problem_type_hard.csv", index_col=0)

In [4]:
df

,index,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
0,0,Не работает блок питания,В течении недели перестал внезапно работать бл...,Ноутбук,Блок питания,C223100360
1,1,Не включается SILA LLC HK2-3404.,Добрый день!_x000D_\nПодскажите пожалуйста что...,Ноутбук,Материнская плата,C223094534
2,2,Сервер СР2-5422 // Системные ошибки // D251110041,Добрый вечер! На сервере появились системные о...,Сервер,Материнская плата,D251110041
3,3,Повреждение матрицы,Прошу произвести диагностику и сориентировать ...,Ноутбук,Матрица,C223014328
4,4,sila HK2-1404 c223014125 asset 7101087,Добрый день!\nПрошу взять ноутбук sila HK2-140...,Ноутбук,Материнская плата,C223014125
...,...,...,...,...,...,...
189,189,НК2-1404 // Не работает Wi-FI (Кашира)// C2231...,На устройстве не работает WiFi\n,Ноутбук,Wi-fi модуль,C223101219
190,190,Не работает блок питания (зарядка),Не работает блок питания (зарядка),Ноутбук,Блок питания,С223093223
191,191,Ноутбук НК2-1404 //Не работает клавиша F1 или ...,"Добрый день,\nпосле ремонта ноутбука перестала...",Ноутбук,Клавиатура,С223010731
192,192,Отказ блока питания,Добрый день! \n У нас перестал работать зарядн...,Ноутбук,Блок питания,C222091750


In [78]:
df_problem_type.rename({"result": "Описание"}, axis=1, inplace=True)

In [80]:
df_problem_type["Тема"] = ""

In [81]:
df_problem_type.to_csv("tmp.csv")

In [84]:
df["Описание"].tolist()

['В течении недели перестал внезапно работать блок питания на ноутбуке  Сила НК2-1404 v2 SN:  C223100360\n',
 'Добрый день!_x000D_\nПодскажите пожалуйста что может, имею ПК SILA LLC HK2-3404 №C223094534._x000D_\nСела батарейка при работе в автономном режиме, поставил на зарядку нажал на запуск запустился а потом выключился и больше не включается._x000D_\nИ не понятно идёт заряд или нет, что можно сделать?_x000D_\n',
 'Добрый вечер! На сервере появились системные ошибки. Скрин и логи прилагаю._x000D_\nПредварительно на 03.09.24 (совместно с работами по замене серверов) инженерам необходимо по возможности устранить проблему или согласовать новую платформу.',
 'Прошу произвести диагностику и сориентировать по условиям ремонта данного ноутбука. \n',
 'Добрый день!\nПрошу взять ноутбук sila HK2-1404 c223014125 asset 7101087 в гарантийный ремонт\nНе включается, нет индикации заряда.\n',
 'Не работает блок питания на ноутбуке sn C223010123\n',
 'У нас же третий из 50 ноутбуков СИЛА проявляет 

In [87]:
set(df["Точка отказа"])

{'Jack',
 'SFP модуль',
 'Wi-fi антенна',
 'Wi-fi модуль',
 'Аккумулятор',
 'Блок питания',
 'Вентилятор',
 'Динамики',
 'Диск',
 'Диск ',
 'Камера',
 'Клавиатура',
 'Консультация',
 'Корпус',
 'Материнская плата',
 'Матрица',
 'Оперативная память',
 'Программное обеспечение',
 'Сервер'}

In [25]:
df["Тема"] = df["Тема"].map(lambda x: re.sub(r'\s+', ' ', x.replace("//", "")).strip())

In [26]:
df["Описание"] = df["Описание"].map(lambda x: re.sub(r'\s+', ' ', x.replace("\n", " ").replace("_x000D_", "").strip()))

In [27]:
df["full_text"] = df["Тема"] + " " + df["Описание"]

In [28]:
texts1, y1 = df["full_text"].tolist(), df["Точка отказа"].tolist()

In [29]:
texts2, y2 = df_problem_type["result"].tolist(), df_problem_type["target"].tolist()

In [30]:
texts = texts1 + texts2
y = y1 + y2

In [31]:
y = [s.strip() for s in y]

In [121]:
def text_lemmatizing(text):
    MAX_WORDS_COUNT = 10000000
    if text is None:
        return " "
    text = str(text)
    if len(text) > MAX_WORDS_COUNT:
        return text
    if text is None: return None
    text = text.replace("\n", " ")
    text = re.sub('[0-9:,\.!?()-/+*;•$&%]', '', text.lower())
    text = " ".join(
        [morph.parse(word)[0].normal_form for word in text.split()[:MAX_WORDS_COUNT]])
    return text

In [33]:
texts = list(map(text_lemmatizing, texts))

In [34]:
from gensim.models import Word2Vec
model_comp_name = Word2Vec.load("api/models/wv2_lematized_16.model")


def get_emb_by_modele(model, comp_names_without_job, vector_size=64):
    all_tokens = set(model.wv.index_to_key)
    word_vectors_dict = {word: model.wv[word] for word in model.wv.index_to_key}

    all_embds = []

    for sent in tqdm(comp_names_without_job):
        all_emb = [word_vectors_dict[word] for word in sent if word in all_tokens]

        if len(all_emb) == 0:
            emb = np.zeros(vector_size)
        else:
            emb = np.mean(all_emb, axis=0)

        all_embds.append(emb)
    emb_comp_name_df = pd.DataFrame(np.array(all_embds), columns=[f"w2v_{i}" for i in range(vector_size)])
    return emb_comp_name_df

In [35]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000).fit(texts)
X = vectorizer.transform(texts)

svd = TruncatedSVD(n_components=256).fit(X)
X = svd.transform(X)

In [36]:
w2v_embeds = get_emb_by_modele(model_comp_name, texts, vector_size=16)

100%|████████████████████████████████████| 5194/5194 [00:00<00:00, 26429.90it/s]


In [38]:
X = pd.concat([pd.DataFrame(X), w2v_embeds], axis=1)

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [41]:
clf = CatBoostClassifier(n_estimators=400, l2_leaf_reg=12, auto_class_weights="Balanced",
                        ).fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50)

0:	learn: 2.8465579	test: 2.8474320	best: 2.8474320 (0)	total: 116ms	remaining: 46.3s
1:	learn: 2.7968474	test: 2.7969058	best: 2.7969058 (1)	total: 230ms	remaining: 45.8s
2:	learn: 2.7469830	test: 2.7505283	best: 2.7505283 (2)	total: 358ms	remaining: 47.3s
3:	learn: 2.7153007	test: 2.7219335	best: 2.7219335 (3)	total: 466ms	remaining: 46.2s
4:	learn: 2.6693150	test: 2.6762935	best: 2.6762935 (4)	total: 581ms	remaining: 45.9s
5:	learn: 2.6119183	test: 2.6200802	best: 2.6200802 (5)	total: 699ms	remaining: 45.9s
6:	learn: 2.5551946	test: 2.5644961	best: 2.5644961 (6)	total: 805ms	remaining: 45.2s
7:	learn: 2.5161945	test: 2.5289242	best: 2.5289242 (7)	total: 922ms	remaining: 45.2s
8:	learn: 2.4638744	test: 2.4801275	best: 2.4801275 (8)	total: 1.04s	remaining: 45.3s
9:	learn: 2.4151078	test: 2.4347714	best: 2.4347714 (9)	total: 1.15s	remaining: 44.9s
10:	learn: 2.3882368	test: 2.4090223	best: 2.4090223 (10)	total: 1.29s	remaining: 45.7s
11:	learn: 2.3532505	test: 2.3757449	best: 2.3757449

94:	learn: 0.8973710	test: 0.9770254	best: 0.9770254 (94)	total: 11.8s	remaining: 38s
95:	learn: 0.8906820	test: 0.9702696	best: 0.9702696 (95)	total: 12s	remaining: 38s
96:	learn: 0.8829132	test: 0.9626631	best: 0.9626631 (96)	total: 12.1s	remaining: 37.8s
97:	learn: 0.8759134	test: 0.9554928	best: 0.9554928 (97)	total: 12.2s	remaining: 37.7s
98:	learn: 0.8688564	test: 0.9487400	best: 0.9487400 (98)	total: 12.4s	remaining: 37.7s
99:	learn: 0.8628526	test: 0.9431018	best: 0.9431018 (99)	total: 12.5s	remaining: 37.5s
100:	learn: 0.8548557	test: 0.9353627	best: 0.9353627 (100)	total: 12.7s	remaining: 37.5s
101:	learn: 0.8489422	test: 0.9305605	best: 0.9305605 (101)	total: 12.8s	remaining: 37.3s
102:	learn: 0.8421940	test: 0.9233932	best: 0.9233932 (102)	total: 12.9s	remaining: 37.2s
103:	learn: 0.8377790	test: 0.9195454	best: 0.9195454 (103)	total: 13s	remaining: 37.1s
104:	learn: 0.8302176	test: 0.9123348	best: 0.9123348 (104)	total: 13.2s	remaining: 37s
105:	learn: 0.8243457	test: 0.90

186:	learn: 0.5147600	test: 0.6043910	best: 0.6043910 (186)	total: 24.2s	remaining: 27.5s
187:	learn: 0.5129503	test: 0.6027086	best: 0.6027086 (187)	total: 24.3s	remaining: 27.4s
188:	learn: 0.5098404	test: 0.5998726	best: 0.5998726 (188)	total: 24.4s	remaining: 27.3s
189:	learn: 0.5071830	test: 0.5968851	best: 0.5968851 (189)	total: 24.6s	remaining: 27.1s
190:	learn: 0.5052529	test: 0.5949979	best: 0.5949979 (190)	total: 24.7s	remaining: 27s
191:	learn: 0.5029585	test: 0.5928396	best: 0.5928396 (191)	total: 24.8s	remaining: 26.9s
192:	learn: 0.5002603	test: 0.5901329	best: 0.5901329 (192)	total: 25s	remaining: 26.8s
193:	learn: 0.4978030	test: 0.5875872	best: 0.5875872 (193)	total: 25.1s	remaining: 26.6s
194:	learn: 0.4958538	test: 0.5856277	best: 0.5856277 (194)	total: 25.2s	remaining: 26.5s
195:	learn: 0.4931807	test: 0.5829464	best: 0.5829464 (195)	total: 25.4s	remaining: 26.4s
196:	learn: 0.4909112	test: 0.5809893	best: 0.5809893 (196)	total: 25.5s	remaining: 26.3s
197:	learn: 0.

278:	learn: 0.3556113	test: 0.4467055	best: 0.4467055 (278)	total: 36.6s	remaining: 15.9s
279:	learn: 0.3541996	test: 0.4455077	best: 0.4455077 (279)	total: 36.7s	remaining: 15.7s
280:	learn: 0.3529644	test: 0.4442471	best: 0.4442471 (280)	total: 36.9s	remaining: 15.6s
281:	learn: 0.3513085	test: 0.4426639	best: 0.4426639 (281)	total: 37.1s	remaining: 15.5s
282:	learn: 0.3496781	test: 0.4408871	best: 0.4408871 (282)	total: 37.4s	remaining: 15.4s
283:	learn: 0.3483654	test: 0.4396701	best: 0.4396701 (283)	total: 37.6s	remaining: 15.4s
284:	learn: 0.3469462	test: 0.4382117	best: 0.4382117 (284)	total: 37.9s	remaining: 15.3s
285:	learn: 0.3462254	test: 0.4376189	best: 0.4376189 (285)	total: 38.1s	remaining: 15.2s
286:	learn: 0.3448913	test: 0.4362979	best: 0.4362979 (286)	total: 38.3s	remaining: 15.1s
287:	learn: 0.3434416	test: 0.4348537	best: 0.4348537 (287)	total: 38.5s	remaining: 15s
288:	learn: 0.3421450	test: 0.4335995	best: 0.4335995 (288)	total: 38.6s	remaining: 14.8s
289:	learn: 

370:	learn: 0.2747660	test: 0.3665879	best: 0.3665879 (370)	total: 50.7s	remaining: 3.97s
371:	learn: 0.2738036	test: 0.3655927	best: 0.3655927 (371)	total: 50.9s	remaining: 3.83s
372:	learn: 0.2730266	test: 0.3649152	best: 0.3649152 (372)	total: 51.1s	remaining: 3.69s
373:	learn: 0.2727771	test: 0.3646434	best: 0.3646434 (373)	total: 51.2s	remaining: 3.56s
374:	learn: 0.2725970	test: 0.3645206	best: 0.3645206 (374)	total: 51.3s	remaining: 3.42s
375:	learn: 0.2717466	test: 0.3637111	best: 0.3637111 (375)	total: 51.5s	remaining: 3.29s
376:	learn: 0.2716201	test: 0.3635927	best: 0.3635927 (376)	total: 51.6s	remaining: 3.15s
377:	learn: 0.2714543	test: 0.3634395	best: 0.3634395 (377)	total: 51.8s	remaining: 3.01s
378:	learn: 0.2713250	test: 0.3632687	best: 0.3632687 (378)	total: 51.9s	remaining: 2.88s
379:	learn: 0.2703218	test: 0.3622225	best: 0.3622225 (379)	total: 52.1s	remaining: 2.74s
380:	learn: 0.2694846	test: 0.3614432	best: 0.3614432 (380)	total: 52.2s	remaining: 2.6s
381:	learn:

In [42]:
clf.score(X_valid, y_valid)

0.9528392685274302

In [43]:
f1_score(clf.predict(X_valid), y_valid, average="weighted")

0.952768440964199